In [1]:
!pip install selenium

     |████████████████████████████████| 911kB 3.5MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install chromedriver-binary

  Stored in directory: /Users/fukuikota/Library/Caches/pip/wheels/30/27/2f/53691e12e1f4a1409f8893d76ea42ec737691a87ded39bbd66
Successfully built chromedriver-binary
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
!pip install mecab-python3

     |████████████████████████████████| 14.1MB 1.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
!brew install mecab
!brew install mecab-ipadic

==> Downloading https://homebrew.bintray.com/bottles/mecab-0.996.high_sierra.bot
==> Downloading from https://akamai.bintray.com/d4/d48340df17075e4a6237ffb87306a
######################################################################## 100.0%
==> Pouring mecab-0.996.high_sierra.bottle.3.tar.gz
🍺  /usr/local/Cellar/mecab/0.996: 20 files, 4.2MB
==> Downloading https://homebrew.bintray.com/bottles/mecab-ipadic-2.7.0-20070801
==> Downloading from https://akamai.bintray.com/ef/ef5cf167b05fd74457d5c31a46750
######################################################################## 100.0%
==> Pouring mecab-ipadic-2.7.0-20070801.high_sierra.bottle.tar.gz
==> Caveats
To enable mecab-ipadic dictionary, add to /usr/local/etc/mecabrc:
  dicdir = /usr/local/lib/mecab/dic/ipadic
==> Summary
🍺  /usr/local/Cellar/mecab-ipadic/2.7.0-20070801: 16 files, 50.6MB


In [3]:
!brew install chromedriver

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> New Formulae
atlantis                                 ipopt
catch2                                   lizard
cjson                                    molten-vk
clojure-lsp                              newman
clzip                                    pprint
csvq                                     procs
dbmate                                   scdoc
deno                                     spice-gtk
docker-machine-driver-vmware             spirv-tools
dust                                     termshark
erlang@21                                terraform@0.11
heatshrink                               terraformer
hey                                      virgil
include-what-you-use
==> Updated Formulae
git ✔                                    mailutils
node ✔                                   mame
sphinx-doc ✔                             mariadb-connector-c
acpica                                   mat2
aide      


Error: No available formula with the name "chromedriver" 
It was migrated from homebrew/core to homebrew/cask.
You can access it again by running:
  brew tap homebrew/cask
And then you can install it by running:
  brew cask install chromedriver


In [6]:
# append import path
import os, sys
print(os.getcwd())
sys.path.append('../')

/Users/fukuikota/Documents/work/ymn/ymn-proken/notebook


In [10]:
import os, sys
from model  import estimation_silent
import time
from selenium import webdriver



driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('https://ikujipapa.net/archives/16621')
result = driver.find_elements_by_tag_name('p')
result_parents = set()
for r in result:
    result_parents.add(r.find_element_by_xpath('..'))

for r in result_parents:
    print(r.get_attribute("id"))
    
print([r.text for r in result])
driver.quit()

ModuleNotFoundError: No module named 'MeCab'

In [ ]:
import time
from selenium import webdriver

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('https://ikujipapa.net/archives/16621')
results_raw = driver.find_elements_by_tag_name('p')


for r in results_raw:
    print(r.get_attribute("id"))
    
print([r.text for r in results])
driver.quit()

In [ ]:
MODEL_PATH = '../data/processing/20190526_名詞のみ.bin'
CONTENT = 'ここにテキストが入ります。'

# -*- coding: utf-8 -*-

import sys
import subprocess as cmd
import MeCab
import fasttext as ft
from sklearn.metrics import mean_squared_error


def text2bow(obj, mod):

    # input: ファイルの場合はmod="file", input: 文字列の場合はmod="str"

    """
    if mod == "file":
        morp = cmd.getstatusoutput("cat " + obj + " | mecab -Owakati")
    elif mod == "str":
        morp = cmd.getstatusoutput("echo " + obj + " | mecab -Owakati")
    else:
        print ("error!!")
        sys.exit(0)
    """

    mecab = MeCab.Tagger("-Ochasen")
    morp = mecab.parse(obj)

    # print(morp)

    words = morp[1]
    words = words.replace('\n','')

    return words

def Scoring(prob):

    score = 0.0
    for e in prob.keys():
        score += e*prob[e]

    return score


def SentimentEstimation(input_txt, clf):

    prob = {}

    bow = text2bow(input_txt, mod="str")

    # print(bow)
    estimate = clf.predict_proba(texts=[bow], k=5)[0]

    for e in estimate:
        index = int(e[0][9])
        prob[index] = e[1]

    score = Scoring(prob)
    return score


def main(model):

    score = SentimentEstimation(CONTENT, model)
    print(score)


if __name__ == "__main__":

    argvs = sys.argv

    _usage = """--                                                                                                                                                       
Usage:                                                                                                                                                                   
    python estimation.py [model]                                                                                                                                         
Args:                                                                                                                                                                    
    [model]: The argument is a model for sentiment estimation that is trained by fastText.                                                                               
""".rstrip()

    if len(argvs) < 2:
        print (_usage)
        sys.exit(0)

    model = ft.load_model(MODEL_PATH)

    main(model)